In [1]:
import pandas as pd

In [1]:
dir = "I:/Data_for_practice/Rfiles/QuanTech"

In [ ]:
dat = pd.read_csv(f"{dir}/quant_pub_ed_eu.csv", index_col=0)
dat = dat.loc[:,['pubid','abstract','itemtitle']]

,pubid,abstract,itemtitle
1,5336454.0,\r\n<p>There are a number of systems that are ...,The control of a superconducting qubit in the ...
2,8064916.0,\r\n<p>The method of stimulated Raman adiabati...,Coherent manipulation of superconducting quant...
3,9565513.0,\r\n<p>We analyze the interaction of a two-lev...,Influence of the asymmetry of the potential on...
4,13480802.0,\r\n<p>A scheme for fine tuning of quantum ope...,Fine tuning of quantum operations performed vi...
5,52091.0,\r\n<p>The dc-superconducting quantum interfer...,Low noise multiwasher superconducting interfer...
...,...,...,...
94890,46588564.0,\r\n<p>As quantum processors grow in complexit...,Scaling silicon-based quantum computing using ...
94891,46593978.0,\r\n<p>We experimentally study a circuit quant...,Quantum versus classical regime in circuit qua...
94892,46598962.0,\r\n<p>SNF1-Related protein kinases Type 2 (Sn...,The SnRK2.10 kinase mitigates the adverse effe...
94893,46599522.0,\r\n<p>We study thermalization and spectral pr...,Thermalization dynamics and spectral statistic...


In [9]:
import ollama
import pandas as pd
import re

desiredModel = 'llama4:scout' # 'llama3.3'

def quant_class(title: str, abstract: str) -> str:
    prompt = f"""
    Below is the title and abstract of an academic paper. Determine whether this paper is related to *quantum science*. 
    Respond clearly with either "Related" or "Not Related". Briefly explain your reasoning.

    Title: {title}

    Abstract: {abstract}


    Answer format:
    - Relevance: Related / Not Related
    - Reason: (brief explanation)
    """
    return prompt

dat = pd.read_csv(f"{dir}/quant_pub_ed_eu.csv", index_col=0)
dat = dat.loc[:,['pubid','abstract','itemtitle']]
dat = dat[dat['abstract'].notnull()]
dat = dat.drop_duplicates()
dat = dat.reset_index(drop=True)
dat = dat.iloc[0]

results = []
title = dat['itemtitle']
abstract = dat['abstract']
abstract = abstract.replace('\r', '').replace('\n', ' ')
abstract = re.sub(r'</?p>', '', abstract, flags=re.IGNORECASE)
abstract = re.sub(r'\s+', ' ', abstract).strip()

prompt = quant_class(title, abstract)

response = ollama.chat(model=desiredModel, messages=[{'role': 'user', 'content': prompt}])
content = response['message']['content']

relevance_match = re.search(r"Relevance:\s*(Related|Not Related)", content, re.IGNORECASE)
reason_match = re.search(r"Reason:\s*(.*)", content, re.IGNORECASE)

relevance = relevance_match.group(1) if relevance_match else "Unclear"
reason = reason_match.group(1).strip() if reason_match else content.strip()

results.append({
    'pubid': dat['pubid'],
    'title': title,
    'abstract': abstract,
    'relevance': relevance,
    'reason': reason
})

df_results = pd.DataFrame(results)
# df_results.to_csv('quantum_classification_results.csv', index=False)
df_results.head()

,pubid,title,abstract,relevance,reason
0,5336454.0,The control of a superconducting qubit in the ...,There are a number of systems that are current...,Related,"The paper discusses superconducting qubits, qu..."


In [ ]:
import ollama
import pandas as pd
import re

desiredModel = 'llama3.3'

def quant_class(title: str, abstract: str) -> str:
    prompt = f"""
    Below is the title and abstract of an academic paper. Determine whether this paper is related to *quantum science*. 
    Respond clearly with either "Related" or "Not Related". Briefly explain your reasoning.

    Title: {title}

    Abstract: {abstract}


    Answer format:
    - Relevance: Related / Not Related
    - Reason: (brief explanation)
    """
    return prompt

dat = pd.read_csv(f"{dir}/quant_pub_ed_eu.csv", index_col=0)
dat = dat.loc[:,['pubid','abstract','itemtitle']]
dat = dat[dat['abstract'].notnull()]
dat = dat.drop_duplicates()
dat = dat.reset_index(drop=True)

results = []
for i, row in dat.iterrows():
    title = row['itemtitle']
    abstract = row['abstract']
    abstract = abstract.replace('\r', '').replace('\n', ' ')
    abstract = re.sub(r'</?p>', '', abstract, flags=re.IGNORECASE)
    abstract = re.sub(r'\s+', ' ', abstract).strip()

    prompt = quant_class(title, abstract)

    response = ollama.chat(model=desiredModel, messages=[{'role': 'user', 'content': prompt}])
    content = response['message']['content']

    relevance_match = re.search(r"Relevance:\s*(Related|Not Related)", content, re.IGNORECASE)
    reason_match = re.search(r"Reason:\s*(.*)", content, re.IGNORECASE)

    relevance = relevance_match.group(1) if relevance_match else "Unclear"
    reason = reason_match.group(1).strip() if reason_match else content.strip()

    results.append({
        'pubid': row['pubid'],
        'title': title,
        'abstract': abstract,
        'relevance': relevance,
        'reason': reason
    })

    print(f"[{i+1}/{len(dat)}] {relevance} - {title}")

df_results = pd.DataFrame(results)
df_results.to_csv('I:/Data_for_practice/Rfiles/QuanTech/quantum_classification_results.csv', index=False)